In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
data = pd.read_csv('/kaggle/input/titanic/train.csv')
data_to_predict = pd.read_csv('/kaggle/input/titanic/test.csv')

print(data.info())
print('*******')
print(data_to_predict.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
*******
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  


In [3]:
convert_dict = {'Pclass': 'object', 'SibSp': 'object', 'Parch': 'object', 'Embarked': 'object'}
data = data.astype(convert_dict)

In [4]:
convert_dict = {'Pclass': 'object', 'SibSp': 'object', 'Parch': 'object', 'Embarked': 'object'}
data_to_predict = data_to_predict.astype(convert_dict)

In [5]:
target_name = 'Survived'
target = data[target_name]

sub = pd.DataFrame()
sub['PassengerId'] = data_to_predict['PassengerId']  

In [6]:
#dropping Cabin as most of the data is null, dropping Name as it doesn't matter (Name's title might matter which will be explored in future models) 
data.drop(['PassengerId', 'Cabin', 'Name'], axis=1, inplace=True)
data_to_predict.drop(['PassengerId', 'Name', 'Cabin'], axis=1, inplace=True)

In [7]:
for column in data.columns:
    if data[column].isnull().values.any():
#         col_with_nulls.append(column)
        print(('Values missing in {} - {}').format(column, data[column].isnull().sum()))

print('-------------------------------')

for column in data_to_predict.columns:
    if data_to_predict[column].isnull().values.any():
#         test_col_with_nulls.append(column)
        print(('Values missing in {} - {}').format(column, data_to_predict[column].isnull().sum()))  

Values missing in Age - 177
Values missing in Embarked - 2
-------------------------------
Values missing in Age - 86
Values missing in Fare - 1


In [8]:
display(data.groupby(['Sex', 'Pclass'])['Age'].mean())

Sex     Pclass
female  1         34.611765
        2         28.722973
        3         21.750000
male    1         41.281386
        2         30.740707
        3         26.507589
Name: Age, dtype: float64

In [9]:
data['Age'] = data['Age'].fillna(round(data.groupby(['Sex', 'Pclass'])['Age'].transform('mean'), 1))

In [10]:
display(data_to_predict.groupby(['Sex', 'Pclass'])['Age'].mean())

Sex     Pclass
female  1         41.333333
        2         24.376552
        3         23.073400
male    1         40.520000
        2         30.940678
        3         24.525104
Name: Age, dtype: float64

In [11]:
data_to_predict['Age'] = data_to_predict['Age'].fillna(round(data_to_predict.groupby(['Sex', 'Pclass'])['Age'].transform('mean'), 1))

In [12]:
data['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [13]:
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])

In [14]:
data_to_predict[data_to_predict['Fare'].isnull()]

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
152,3,male,60.5,0,0,3701,NaN,S


In [15]:
data_to_predict['Fare'] = data_to_predict['Fare'].fillna(round(data_to_predict.groupby(['Sex', 'Pclass'])['Fare'].transform('mean'), 4))

In [16]:
for column in data.columns:
    if data[column].isnull().values.any():
#         col_with_nulls.append(column)
        print(('Values missing in {} - {}').format(column, data[column].isnull().sum()))

print('-------------------------------')

for column in data_to_predict.columns:
    if data_to_predict[column].isnull().values.any():
#         test_col_with_nulls.append(column)
        print(('Values missing in {} - {}').format(column, data_to_predict[column].isnull().sum()))  

-------------------------------


In [17]:
display(data.info())
display(data_to_predict.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    object 
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    object 
 5   Parch     891 non-null    object 
 6   Ticket    891 non-null    object 
 7   Fare      891 non-null    float64
 8   Embarked  891 non-null    object 
dtypes: float64(2), int64(1), object(6)
memory usage: 62.8+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    object 
 1   Sex       418 non-null    object 
 2   Age       418 non-null    float64
 3   SibSp     418 non-null    object 
 4   Parch     418 non-null    object 
 5   Ticket    418 non-null    object 
 6   Fare      418 non-null    float64
 7   Embarked  418 non-null    object 
dtypes: float64(2), object(6)
memory usage: 26.2+ KB


None

In [18]:
data.drop(['Survived'], axis=1, inplace=True)

In [19]:
from sklearn.compose import make_column_selector as selector

num_col_selector = selector(dtype_exclude=object)
cat_col_selector = selector(dtype_include=object)

num_feat = num_col_selector(data)
cat_cols = cat_col_selector(data)

print(cat_cols)

['Pclass', 'Sex', 'SibSp', 'Parch', 'Ticket', 'Embarked']


In [20]:
for col in ('Pclass', 'SibSp', 'Parch'):
    print(f'({col}, {sorted(data[col].unique())}),')

(Pclass, [1, 2, 3]),
(SibSp, [0, 1, 2, 3, 4, 5, 8]),
(Parch, [0, 1, 2, 3, 4, 5, 6]),


In [21]:
ordered_ordinal_feat = [('Pclass', [3, 2, 1]),
                        ('SibSp', [0, 1, 2, 3, 4, 5, 8]),
                        ('Parch', [0, 1, 2, 3, 4, 5, 6, 9])]

ord_feat, ord_cats = zip(*ordered_ordinal_feat)
ord_feat, ord_cats = list(ord_feat), list(ord_cats)
nom_feat = list(set(cat_cols) - set(ord_feat))

cat_feat = ord_feat + nom_feat

X = data[num_feat + cat_feat]
to_predict = data_to_predict[num_feat + cat_feat]

num_feat_end = len(num_feat)
ord_feat_end = num_feat_end + len(ord_feat)
nom_feat_end = ord_feat_end + len(nom_feat)

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.base import clone

scalar = RobustScaler(with_centering=False)
ohe = OneHotEncoder(handle_unknown='ignore')
ordi = OrdinalEncoder(categories=ord_cats)

cat_trf = ColumnTransformer([
    ('Nominal', ohe, [x for x in range(ord_feat_end, nom_feat_end)]),
    ('Ordinal', ordi, [x for x in range(num_feat_end, ord_feat_end)])
], remainder='passthrough')

num_trf = ColumnTransformer([
    ('Scaling', scalar, [x for x in range(ord_feat_end)])
], remainder='passthrough')

preprocessor = make_pipeline(cat_trf, num_trf)

# data_train, data_test, target_train, target_test = train_test_split(
#     X, target, test_size=0.1, random_state=42)

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, make_scorer 
import warnings
warnings.filterwarnings("ignore")

rn=24
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=rn)

model_RF = make_pipeline(clone(preprocessor), RandomForestClassifier(class_weight="balanced"))

param_RF= {"randomforestclassifier__max_features":np.arange(1,60), 
                "randomforestclassifier__n_estimators": np.arange(150,250,10),
                "randomforestclassifier__min_samples_leaf":[1, 3, 7, 10]}

RF= RandomizedSearchCV(estimator=model_RF, param_distributions=param_RF , scoring=make_scorer(accuracy_score), cv=kfold, n_iter=250, n_jobs=-1, verbose=1)
RF.fit(X,target)
RF_best = RF.best_estimator_
RF.best_score_, RF.best_params_

Fitting 10 folds for each of 250 candidates, totalling 2500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   38.0s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 2500 out of 2500 | elapsed:  7.8min finished


(0.844019975031211,
 {'randomforestclassifier__n_estimators': 200,
  'randomforestclassifier__min_samples_leaf': 1,
  'randomforestclassifier__max_features': 23})

In [24]:
# rsRF.score(data_test, target_test)

In [25]:
from sklearn.neighbors import KNeighborsClassifier

model_KNN = make_pipeline(clone(preprocessor), KNeighborsClassifier())

KNN_param = {'kneighborsclassifier__n_neighbors': [5, 10, 15, 18, 20, 25, 28, 30, 32, 35, 40, 45, 50],
             'kneighborsclassifier__weights': ['uniform', 'distance'],
             'kneighborsclassifier__algorithm': ['brute'],
             'kneighborsclassifier__leaf_size': [23, 25, 27, 30, 20, 35, 40]}

KNN = RandomizedSearchCV(model_KNN, KNN_param, scoring=make_scorer(accuracy_score), cv=kfold, n_jobs=-1, verbose=1)
KNN.fit(X, target)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.4s finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=24, shuffle=True),
                   estimator=Pipeline(steps=[('pipeline',
                                              Pipeline(steps=[('columntransformer-1',
                                                               ColumnTransformer(remainder='passthrough',
                                                                                 transformers=[('Nominal',
                                                                                                OneHotEncoder(handle_unknown='ignore'),
                                                                                                [5,
                                                                                                 6,
                                                                                                 7]),
                                                                                               ('Ordinal',
               

In [26]:
print(KNN.best_score_)
print(KNN.best_params_)
# print(KNN.score(data_test, target_test))

0.7407490636704119
{'kneighborsclassifier__weights': 'distance', 'kneighborsclassifier__n_neighbors': 25, 'kneighborsclassifier__leaf_size': 20, 'kneighborsclassifier__algorithm': 'brute'}


In [27]:
from catboost import CatBoostClassifier

model_CB = make_pipeline(clone(preprocessor), CatBoostClassifier(verbose=0))

CB_param= {'catboostclassifier__depth': [2,3,4,5,6,7,8],
           'catboostclassifier__learning_rate':[0.01, 0.05, 0.1, 0.2, 0.3, 0.4],
           'catboostclassifier__n_estimators':[50,100,200,300]}

# rsCB= RandomizedSearchCV(estimator=CB, param_distributions=param_grid_CB, scoring=make_scorer(score_fnc),cv=kfold, n_iter=250, n_jobs=-1, verbose=1)

CB = RandomizedSearchCV(model_CB, CB_param, scoring=make_scorer(accuracy_score), cv=kfold, n_jobs=-1, verbose=1)
CB.fit(X,target)
CB_best = CB.best_estimator_
CB.best_score_, CB.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   36.7s finished


(0.8428714107365793,
 {'catboostclassifier__n_estimators': 300,
  'catboostclassifier__learning_rate': 0.2,
  'catboostclassifier__depth': 7})

In [28]:
from sklearn.ensemble import GradientBoostingClassifier

model_GBC = make_pipeline(clone(preprocessor), GradientBoostingClassifier())

GBC_param = {'gradientboostingclassifier__loss':["deviance","exponential"],
             'gradientboostingclassifier__n_estimators' : [300,400,500],
             'gradientboostingclassifier__learning_rate': [0.2, 0.1, 0.05, 0.01, 0.001],
             'gradientboostingclassifier__max_depth': [2,4,8,16],
             'gradientboostingclassifier__min_samples_leaf': [25,50,100,150,250],
             'gradientboostingclassifier__max_features': [0.7,0.5, 0.3, 0.1]}

GBC = RandomizedSearchCV(model_GBC, GBC_param, scoring=make_scorer(accuracy_score), n_iter=250, cv=kfold, n_jobs=-1, verbose=1)
GBC.fit(X,target)
GBC_best = GBC.best_estimator_
GBC.best_score_, GBC.best_params_

Fitting 10 folds for each of 250 candidates, totalling 2500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   29.5s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 2500 out of 2500 | elapsed:  7.7min finished


(0.8360923845193508,
 {'gradientboostingclassifier__n_estimators': 400,
  'gradientboostingclassifier__min_samples_leaf': 25,
  'gradientboostingclassifier__max_features': 0.7,
  'gradientboostingclassifier__max_depth': 16,
  'gradientboostingclassifier__loss': 'exponential',
  'gradientboostingclassifier__learning_rate': 0.01})

In [29]:
from sklearn.ensemble import VotingClassifier

VotingC = VotingClassifier(estimators=[('RF', RF_best), ('CB', CB_best), ('GBC', GBC_best)], voting='hard', n_jobs=-1)
VotingC = VotingC.fit(X, target)

In [30]:
models = [RF, KNN, CB, GBC, VotingC]
model_names = ['RF', 'KNN', 'CB', 'GBC', 'VC']

In [31]:
for i, m in enumerate(models):
    out = sub
    out[target_name] = m.predict(to_predict)
    out.to_csv(f'submission {model_names[i]}.csv', index=False)

Best Submssion Accuracy Score is 0.78947 which was given by VotingClassifier (top 8.9% at the time).